In [3]:
import pandas as pd
# EMBEDDINGS_DF_PATH = "Pickled_Data/Embeddings_resnet_retrained_lastLayer/"
# EMBEDDINGS_DF_PATH_REFORMATTED = "Pickled_Data/Embeddings_reformatted/"
EMBEDDING_PATHS = ["Pickled_Data/Embeddings_resnet_retrained_lastLayer/", "Pickled_Data/Embeddings_final/"]
dir_contents = os.listdir(EMBEDDING_PATHS[0])
dir_contents.sort()
dir_contents

['resnet50_no_label_smoothing_model_100.pkl',
 'resnet50_no_label_smoothing_model_125.pkl',
 'resnet50_no_label_smoothing_model_150.pkl',
 'resnet50_no_label_smoothing_model_25.pkl',
 'resnet50_no_label_smoothing_model_50.pkl',
 'resnet50_no_label_smoothing_model_75.pkl',
 'resnet50_removed_mixup_cutmix_model_100.pkl',
 'resnet50_removed_mixup_cutmix_model_150.pkl',
 'resnet50_removed_mixup_cutmix_model_200.pkl',
 'resnet50_removed_mixup_cutmix_model_245.pkl',
 'resnet50_removed_mixup_cutmix_model_50.pkl']

In [4]:
PKL_LOAD_NAMES = [
 'resnet50_no_label_smoothing_model_25.pkl',
 'resnet50_no_label_smoothing_model_50.pkl',
 'resnet50_no_label_smoothing_model_75.pkl',
 'resnet50_no_label_smoothing_model_100.pkl',
 'resnet50_no_label_smoothing_model_125.pkl',
 'resnet50_no_label_smoothing_model_150.pkl'
 ]

EMBEDDING_NAMES = [pkl_name.split(".")[0] for pkl_name in PKL_LOAD_NAMES]

embeddings_dict = {}
for embedding_df in PKL_LOAD_NAMES:
    for embedding_folder in EMBEDDING_PATHS:
        embeddings_path = embedding_folder + embedding_df
        if os.path.exists(embeddings_path):
            break
    else:
        raise ValueError(f"{embedding_df} not found in {EMBEDDING_PATHS}")
    embeddings_dict[embedding_df.split(".")[0]] = pd.read_pickle(embeddings_path)
    print(f"Loaded {embedding_df} from {embeddings_path}...")



Loaded resnet50_no_label_smoothing_model_25.pkl from Pickled_Data/Embeddings_resnet_retrained_lastLayer/resnet50_no_label_smoothing_model_25.pkl...
Loaded resnet50_no_label_smoothing_model_50.pkl from Pickled_Data/Embeddings_resnet_retrained_lastLayer/resnet50_no_label_smoothing_model_50.pkl...
Loaded resnet50_no_label_smoothing_model_75.pkl from Pickled_Data/Embeddings_resnet_retrained_lastLayer/resnet50_no_label_smoothing_model_75.pkl...
Loaded resnet50_no_label_smoothing_model_100.pkl from Pickled_Data/Embeddings_resnet_retrained_lastLayer/resnet50_no_label_smoothing_model_100.pkl...
Loaded resnet50_no_label_smoothing_model_125.pkl from Pickled_Data/Embeddings_resnet_retrained_lastLayer/resnet50_no_label_smoothing_model_125.pkl...
Loaded resnet50_no_label_smoothing_model_150.pkl from Pickled_Data/Embeddings_resnet_retrained_lastLayer/resnet50_no_label_smoothing_model_150.pkl...


In [5]:
# Rename and merge loaded dataframes
dict_items = iter(embeddings_dict.items())
first_key, first_df = next(dict_items)
model_embeddings_df = first_df.copy()
if "file_exists" in model_embeddings_df.columns:
    model_embeddings_df.drop("file_exists", axis=1, inplace=True)
model_embeddings_df.rename({"embeddings":first_key}, axis=1, inplace=True)
for key, item_df in dict_items:
    item_df = pd.DataFrame(item_df[["id", "embeddings"]])
    item_df.rename({"embeddings": key}, axis=1, inplace=True)
    model_embeddings_df = pd.merge(model_embeddings_df, item_df, on="id", how="inner")

# Load outfit tags and filter unused categories
PICKLED_DATA_FOLDER = "Pickled_Data/"
PICKLED_OUTFITS_FILE = "active_outfits_df.pkl"
OUTFITS_DROP_COLUMNS = ['owner', 'name', 'brand', 'isPublic', 'isDeleted', 'meta.validFrom', 'meta.validTo', "Outfit_size"]

outfits_df = pd.read_pickle(PICKLED_DATA_FOLDER + PICKLED_OUTFITS_FILE)
outfits_df.drop(OUTFITS_DROP_COLUMNS, axis=1, inplace=True)
#IGNORE_CATEGORIES = ["Size", "Brand", "brand", "Fit"]
# "brand" is an invalid category name, actual brand names are in "Brand"
IGNORE_CATEGORIES = ["Size", "brand", "Fit", "Shoe Size"]
outfits_df["filtered_tags"] = outfits_df.apply(lambda row: [tag for tag, cat in zip(row["outfit_tags"], row["tag_categories"]) if cat not in IGNORE_CATEGORIES], axis=1)
outfits_df["filtered_tag_categories"] = outfits_df.apply(lambda row: [cat for cat in row["tag_categories"] if cat not in IGNORE_CATEGORIES], axis=1)
outfits_df["filtered_tags"].explode().value_counts()

Everyday          6769
Spring            3973
Fall              3149
Summer            3037
Long arms         2664
                  ... 
Summum Woman         1
Saboo                1
Surface to Air       1
Osklen               1
GAP                  1
Name: filtered_tags, Length: 540, dtype: int64

In [6]:

TAG_WEIGHT_DF_PATH = f"Pickled_Data/tag_entropy_df.pkl"
TO_DROP_COLUMNS = ["Shoe Size", "brand"]
tag_weights_df = pd.read_pickle(TAG_WEIGHT_DF_PATH)
tag_weights_df.loc["default_weight"] = 1
tag_weights_df.loc["inverse_normalized"] = 1 - tag_weights_df.loc["normalized"]
tag_weights_df.drop(TO_DROP_COLUMNS, axis=1, inplace=True)
tag_weights_df

,Brand,Material,Category,Color,Size,No category,Occassion,Sleeve,Embellishment,Neckline,Waist,Pattern,Fit,Length,Occasion
entropy,2.150160,2.052015,1.293279,1.930027,1.175349,1.933907,1.735647,1.204265,0.602437,1.310849,1.152163,0.902646,1.094322,1.033667,0.140358
normalized,0.348272,0.549009,0.412464,0.655482,0.458235,0.778261,0.753782,0.579129,0.289711,0.630385,0.592095,0.463868,0.679941,0.642253,0.127759
lower,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
upper,6.173786,3.737670,3.135494,2.944439,2.564949,2.484907,2.302585,2.079442,2.079442,2.079442,1.945910,1.945910,1.609438,1.609438,1.098612
default_weight,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
inverse_normalized,0.651728,0.450991,0.587536,0.344518,0.541765,0.221739,0.246218,0.420871,0.710289,0.369615,0.407905,0.536132,0.320059,0.357747,0.872241


In [7]:
tags_list = outfits_df["filtered_tags"].explode().unique()
tags_list

array(['Wool', 'Winter', 'Everyday', 'Fall', 'Cardigans', 'Black',
       'Long arms', 'Spring', 'Round Neck', 'FJONG Bump', 'Collar',
       'FWSS', 'Business', 'FJONG Active', 'Outerwear', 'Normal',
       'Fleischer Couture', 'Polyester', 'Coats', 'Down', 'Green',
       'Cotton', 'Skappel', 'V-neck', 'Cashmere', 'Shirts', 'Blouses',
       'Maud', 'Pink', 'Pattern', 'Tops', 'High', 'Skirts', 'Adjustable',
       'Mini', 'Party', 'Brown', 'Classic', 'Going out', 'Dresses',
       'Holzweiler', 'Dressed-up', 'Boat Neck', 'Summer', 'Linen',
       'Beige', 'Viscose', 'T-shirt', 'Floral', 'Romantic', 'Blue',
       'Purple', 'ILAG', 'Orange', 'Multicolor', 'Midi', 'White',
       'Studio Nicholson', 'Nylon', 'The Sept', 'Cala Jade', 'Leather',
       'Bags', 'Høst & Vår', 'Maxi', 'Grey', 'Mette Møller', 'Ruffles',
       'Creative Collective', 'Denim', 'Elastane', 'Holzweiler x FJONG',
       'Pants', 'Checkers', 'Baum und Pferdgarten', 'Untold Stories',
       'Designers Remix', 'Blaz

In [8]:
# to_dict_df = tag_weights_df.copy()
# to_dict_df.set_index("category", inplace=True)
# mean_weights_dict = to_dict_df["mean_weight"].to_dict()
# default_weights_dict = to_dict_df["default_weight"].to_dict()

#entropy_weights_dict = tag_weights_df.to_dict(orient="index")["entropy"]
#normalized_dict = tag_weights_df.to_dict(orient="index")["normalized"]

# Use normalized and inversed entropy weights as well as default weights
inverse_normalized_dict = tag_weights_df.to_dict(orient="index")["inverse_normalized"]
default_weights_dict = tag_weights_df.to_dict(orient="index")["default_weight"]
inverse_normalized_dict

{'Brand': 0.6517275329005947,
 'Material': 0.45099088486520245,
 'Category': 0.5875357573816247,
 'Color': 0.3445177815044965,
 'Size': 0.5417652895026976,
 'No category': 0.2217385295534735,
 'Occassion': 0.24621794294890775,
 'Sleeve': 0.4208708368339431,
 'Embellishment': 0.710288940274548,
 'Neckline': 0.36961496253511683,
 'Waist': 0.4079054911922093,
 'Pattern': 0.5361318428719093,
 'Fit': 0.3200592889226679,
 'Length': 0.3577465213862313,
 'Occasion': 0.8722406628082692}

In [9]:
from sklearn import preprocessing
import numpy as np
import torch

def convert_to_vector(tags, tag_cats, tag_list, weights_dict):
    tag_vector = np.zeros(len(tag_list))
    for tag, cat in zip(tags, tag_cats):
        tag_index = np.where(tag_list == tag)[0][0]
        tag_vector[tag_index] = round(weights_dict[cat] * 1, 2)    
    return torch.Tensor(tag_vector.reshape(1, -1))

outfits_df["tag_vector"] = outfits_df.apply(lambda row: convert_to_vector(row["filtered_tags"], row["filtered_tag_categories"], tags_list, inverse_normalized_dict), axis=1)
outfits_df["tag_vector_binary"] = outfits_df.apply(lambda row: convert_to_vector(row["filtered_tags"], row["filtered_tag_categories"], tags_list, default_weights_dict), axis=1)
outfits_df["tag_vector_random"] = [torch.randint(0,2, size=(1, len(tags_list))) for _ in range(len(outfits_df))]
outfits_df["tag_vector_shuffled"] = np.random.permutation(outfits_df["tag_vector"].values)
outfits_df

,id,outfit_tags,tag_categories,tag_encoding,filtered_tags,filtered_tag_categories,tag_vector,tag_vector_binary,tag_vector_random,tag_vector_shuffled
0,outfit.00004b4d01ca4ab0a70cf073ba74fefa,"[Wool, Winter, Everyday, Fall, Medium, Cardiga...","[Material, No category, Occassion, No category...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Wool, Winter, Everyday, Fall, Cardigans, Blac...","[Material, No category, Occassion, No category...","[[tensor(0.4500), tensor(0.2200), tensor(0.250...","[[tensor(1.), tensor(1.), tensor(1.), tensor(1...","[[tensor(1), tensor(0), tensor(1), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens..."
1,outfit.00234201fa2d4ee49a572d650c775213,"[FJONG Active, Winter, Outerwear, Everyday, No...","[Occassion, No category, Category, Occassion, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[FJONG Active, Winter, Outerwear, Everyday, No...","[Occassion, No category, Category, Occassion, ...","[[tensor(0.), tensor(0.2200), tensor(0.2500), ...","[[tensor(0.), tensor(1.), tensor(1.), tensor(0...","[[tensor(1), tensor(0), tensor(1), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0..."
2,outfit.003356af26964c5084d0cc1e9f95978c,"[Winter, Cotton, Everyday, Fall, Skappel, V-ne...","[No category, Material, Occassion, No category...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Winter, Cotton, Everyday, Fall, Skappel, V-ne...","[No category, Material, Occassion, No category...","[[tensor(0.), tensor(0.2200), tensor(0.2500), ...","[[tensor(0.), tensor(1.), tensor(1.), tensor(1...","[[tensor(1), tensor(0), tensor(0), tensor(0), ...","[[tensor(0.), tensor(0.2200), tensor(0.2500), ..."
3,outfit.003e042f5e9e4043bf10a0284d88ce75,"[Everyday, Normal, Blouses, Long arms, Spring,...","[Occassion, Waist, Category, Sleeve, No catego...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Everyday, Normal, Blouses, Long arms, Spring,...","[Occassion, Waist, Category, Sleeve, No catego...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(1), tensor(1), tensor(1), tensor(0), ...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0..."
4,outfit.005123277cc440ffbac2d52252cfcac1,"[High, Wrap, Skirts, Adjustable, Polyester, Mi...","[Waist, Fit, Category, Waist, Material, Length...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[High, Skirts, Adjustable, Polyester, Mini, Pa...","[Waist, Category, Waist, Material, Length, Occ...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(1), tensor(0), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens..."
...,...,...,...,...,...,...,...,...,...,...
7901,outfit.ffe4d235a016429dae41bb6399cf16d1,"[Accessories, Onesize, Everyday, Kaibosh, Grey...","[Category, Size, Occassion, Brand, Color, Mate...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Accessories, Everyday, Kaibosh, Grey, Acetate]","[Category, Occassion, Brand, Color, Material]","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(1), tensor(1), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0..."
7902,outfit.ffe617f43d244ebc81228915cfcc6c8e,"[Wool, Dressed-up, Large, Elastane, Everyday, ...","[Material, No category, Size, Material, Occass...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Wool, Dressed-up, Elastane, Everyday, Fall, B...","[Material, No category, Material, Occassion, N...","[[tensor(0.4500), tensor(0.), tensor(0.2500), ...","[[tensor(1.), tensor(0.), tensor(1.), tensor(1...","[[tensor(0), tensor(1), tensor(0), tensor(0), ...","[[tensor(0.), tensor(0.2200), tensor(0.2500), ..."
7903,outfit.ffebad2c479045a78adecdcd8f07427d,"[Dressed-up, Høst & Vår, Viscose, Floral, Ever...","[No category, Brand, Material, Pattern, Occass...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Dressed-up, Høst & Vår, Viscose, F

In [10]:
outfit_merge = pd.DataFrame(outfits_df[["id", "tag_vector", "tag_vector_binary", "tag_vector_random", "tag_vector_shuffled"]])
outfit_merge.rename({"id":"owner"}, axis=1, inplace=True)
encodings_df = pd.merge(model_embeddings_df, outfit_merge, on="owner", how="inner")

# Generate random tensor embeddings for each outfit.
for column in EMBEDDING_NAMES:
    encodings_sample_tensor = torch.tensor(np.vstack(encodings_df[column].iloc[:2000].to_list()), dtype=torch.float32)
    abs_encodings_sample_tensor = torch.abs(encodings_sample_tensor)
    print(column, encodings_sample_tensor.shape, encodings_sample_tensor.max(), encodings_sample_tensor.min(), encodings_sample_tensor.mean(), abs_encodings_sample_tensor.mean())

arb_tensor_shape = encodings_df[EMBEDDING_NAMES[0]].iloc[0].shape
generated_tensor = torch.rand(arb_tensor_shape)
print(f"Generating random embeddings with same shape as {EMBEDDING_NAMES[0]}({arb_tensor_shape})")

encodings_df["random"] = [torch.rand(arb_tensor_shape) for _ in range(len(encodings_df))]
encodings_df["random_shuffle"] = np.random.permutation(encodings_df[EMBEDDING_NAMES[0]].values)
encodings_df

resnet50_no_label_smoothing_model_25 torch.Size([2000, 2048]) tensor(6.7266) tensor(0.) tensor(0.0270) tensor(0.0270)
resnet50_no_label_smoothing_model_50 torch.Size([2000, 2048]) tensor(5.4531) tensor(0.) tensor(0.0299) tensor(0.0299)
resnet50_no_label_smoothing_model_75 torch.Size([2000, 2048]) tensor(7.0898) tensor(0.) tensor(0.0305) tensor(0.0305)
resnet50_no_label_smoothing_model_100 torch.Size([2000, 2048]) tensor(5.7500) tensor(0.) tensor(0.0309) tensor(0.0309)
resnet50_no_label_smoothing_model_125 torch.Size([2000, 2048]) tensor(8.0703) tensor(0.) tensor(0.0275) tensor(0.0275)
resnet50_no_label_smoothing_model_150 torch.Size([2000, 2048]) tensor(9.0391) tensor(0.) tensor(0.0259) tensor(0.0259)
Generating random embeddings with same shape as resnet50_no_label_smoothing_model_25(torch.Size([2048]))


,id,owner,resnet50_no_label_smoothing_model_25,resnet50_no_label_smoothing_model_50,resnet50_no_label_smoothing_model_75,resnet50_no_label_smoothing_model_100,resnet50_no_label_smoothing_model_125,resnet50_no_label_smoothing_model_150,tag_vector,tag_vector_binary,tag_vector_random,tag_vector_shuffled,random,random_shuffle
0,picture.00058abb53434872ae9bb4270ae21f8e,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0.159...","[tensor(0., dtype=torch.float16), tensor(0.124...","[tensor(0., dtype=torch.float16), tensor(0.046...","[tensor(0., dtype=torch.float16), tensor(0.107...","[tensor(0., dtype=torch.float16), tensor(0.155...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(1), tensor(0), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[tensor(0.0449), tensor(0.4612), tensor(0.1682...","[tensor(0., dtype=torch.float16), tensor(0.002..."
1,picture.2911fa95ced4417d8cfced005367bf5b,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(0., dtype=torch.float16), tensor(0.002...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(1), tensor(0), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[tensor(0.4968), tensor(0.7666), tensor(0.6466...","[tensor(0., dtype=torch.float16), tensor(0., d..."
2,picture.3be163f353de4bceaffdebbd22429a80,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(0., dtype=torch.float16), tensor(0.039...","[tensor(0., dtype=torch.float16), tensor(0.011...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0.134...","[tensor(0., dtype=torch.float16), tensor(0.150...","[tensor(0., dtype=torch.float16), tensor(0., d...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(1), tensor(0), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[tensor(0.6954), tensor(0.3622), tensor(0.0382...","[tensor(0., dtype=torch.float16), tensor(0.575..."
3,picture.5ba8cf9fa702483ab6db2a5e39ea24ff,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0.178...","[tensor(0., dtype=torch.float16), tensor(0.044...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0.211...","[tensor(0., dtype=torch.float16), tensor(0., d...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(1), tensor(0), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[tensor(0.9027), tensor(0.7772), tensor(0.9446...","[tensor(0.0396, dtype=torch.float16), tensor(0..."
4,picture.d55b9888fe31446cafd45c64be9b6702,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0.101...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0.039...","[tensor(0., dtype=torch.float16), tensor(0., d...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(1), tensor(0), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[tensor(0.6710), tensor(0.5547), tensor(0.1690...","[tensor(0., dtype=torch.float16), tensor(0.174..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16793,picture.fd54d39a745844db8f640d2e6aca9317,outfit.d5f64f6d2edd4b98a7fb76d114a

In [11]:
from torch.nn import functional as F
from torchmetrics.classification import BinaryJaccardIndex, JaccardIndex
EXAMPLE_TARGET_INDEX = 0
target_vector = encodings_df.iloc[EXAMPLE_TARGET_INDEX]["tag_vector"]
test_vector = encodings_df.iloc[201]["tag_vector"]
jaccard_index = BinaryJaccardIndex()
F.cosine_similarity(target_vector, test_vector)

tensor([0.1694])

In [12]:
from torch.nn import functional as F
from torchmetrics.classification import BinaryJaccardIndex
VECTOR_DISTANCE_POINTS = "tag_vector"
target_vector = encodings_df.iloc[EXAMPLE_TARGET_INDEX][VECTOR_DISTANCE_POINTS]
jaccard_index = BinaryJaccardIndex()
#encodings_df["vector_distance"] = encodings_df["tag_vector"].apply(lambda x: jaccard_index(x, target_vector).item())
encodings_df["vector_distance"] = encodings_df[VECTOR_DISTANCE_POINTS].apply(lambda x: F.cosine_similarity(x, target_vector).item())
#encodings_df["vector_distance"] = encodings_df["vector_distance"].apply(lambda x: 1 - x)
display(encodings_df["vector_distance"].describe())
encodings_df.head()

count    16798.000000
mean         0.212657
std          0.144192
min          0.000000
25%          0.095639
50%          0.191602
75%          0.301925
max          1.000000
Name: vector_distance, dtype: float64

,id,owner,resnet50_removed_mixup_cutmix_model_100,resnet50_removed_mixup_cutmix_model_150,resnet50_removed_mixup_cutmix_model_200,resnet50_removed_mixup_cutmix_model_245,resnet50_removed_mixup_cutmix_model_50,tag_vector,tag_vector_binary,tag_vector_random,tag_vector_shuffled,random,random_shuffle,vector_distance
0,picture.00058abb53434872ae9bb4270ae21f8e,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0.119...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0), tensor(0), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[tensor(0.6224), tensor(0.9694), tensor(0.5404...","[tensor(0., dtype=torch.float16), tensor(0., d...",1.0
1,picture.2911fa95ced4417d8cfced005367bf5b,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(0.0839, dtype=torch.float16), tensor(0...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0.008...","[tensor(0., dtype=torch.float16), tensor(0., d...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0), tensor(0), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[tensor(0.8110), tensor(0.7185), tensor(0.6919...","[tensor(0., dtype=torch.float16), tensor(0., d...",1.0
2,picture.3be163f353de4bceaffdebbd22429a80,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0.0817, dtype=torch.float16), tensor(0...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0.0743, dtype=torch.float16), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0), tensor(0), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[tensor(0.5728), tensor(0.0797), tensor(0.7187...","[tensor(0., dtype=torch.float16), tensor(0., d...",1.0
3,picture.5ba8cf9fa702483ab6db2a5e39ea24ff,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0.089...","[tensor(0., dtype=torch.float16), tensor(0.005...","[tensor(0., dtype=torch.float16), tensor(0., d...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0), tensor(0), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[tensor(0.2569), tensor(0.2582), tensor(0.1983...","[tensor(0., dtype=torch.float16), tensor(0.001...",1.0
4,picture.d55b9888fe31446cafd45c64be9b6702,outfit.98f32aaf08bc4ff09c44e6e11e9199bc,"[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0., d...","[tensor(0., dtype=torch.float16), tensor(0.000...","[tensor(0., dtype=torch.float16), tensor(0.009...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[[tensor(0.), tensor(0.), tensor(1.), tensor(0...","[[tensor(0), tensor(0), tensor(0), tensor(1), ...","[[tensor(0.), tensor(0.), tensor(0.2500), tens...","[tensor(0.7550), tensor(0.0267), tensor(0.7558...","[tensor(0., dtype=torch.float16), tensor(0.000...",1.0


In [13]:
EMBEDDING_NAMES

['resnet50_removed_mixup_cutmix_model_100',
 'resnet50_removed_mixup_cutmix_model_150',
 'resnet50_removed_mixup_cutmix_model_200',
 'resnet50_removed_mixup_cutmix_model_245',
 'resnet50_removed_mixup_cutmix_model_50']

In [14]:
import copy

import wandb
LOG_WANDB = False
if LOG_WANDB:
    wandb.init(project="Image_similarity_tag_correlations", entity="kaborg15")

    WANDB_RUN_NAME = "tag_vector_entropy_1000"
    wandb.run.name = WANDB_RUN_NAME
from tqdm import tqdm

def distance_evaluator_altered(embedding_1, embedding_2):
    return F.cosine_similarity(embedding_1.float(), embedding_2.float(), dim=0).item()
    #return torch.dist(embedding_1, embedding_2, 2).item()

DISTANCE_COLUMN_EXSTENSION = "_distance0"

print("Evaluating distances using hard-coded column subset")
# DISTANCE_COLUMNS_RUN = [
#  ('resnet50_V1_Config', 76.13, 92.862),
#  ('resnet50_V2_Config', 80.858, 95.434),
#  ('resnet50_Init_run_model_27', 46.566, 73.258),
#  ('resnet50_Init_run_model_49', 66.682, 87.802),
#  ('resnet50_V2_Init_run_model_40', 56.598, 81.016),
#  ('resnet50_no_label_smoothing_model_75', 58.474, 82.498),
#  ('resnet50_removed_data_mods_model_64', 26.500, 51.984),
#  ('resnet50_no_data_augs_model_55', 54.940, 80.000),
#  ('resnet50_removed_mixup_cutmix_model_33', 58.840, 82.802)
#  ]
DISTANCE_COLUMNS_RUN = [
 ('resnet50_removed_mixup_cutmix_model_50', 1, 1),
 ('resnet50_removed_mixup_cutmix_model_100', 1, 1),
 ('resnet50_removed_mixup_cutmix_model_150', 1, 1),
 ('resnet50_removed_mixup_cutmix_model_200', 1, 1),
 ('resnet50_removed_mixup_cutmix_model_245', 1, 1)
]

TAG_VECTOR_COLUMNS = ["tag_vector", "tag_vector_binary"]#, "tag_vector_random",	"tag_vector_shuffled"]
MULTIPLICATION_COLUMNS = copy.copy(DISTANCE_COLUMNS_RUN)
MULTIPLICATION_EXTENSION = "_sigmoid"
COLUMN_MULTIPLICATION_FACTOR = 2
VECTOR_DISTANCE_EXTENSION = "_distance"

wandb.config = {
    "distance_columns": DISTANCE_COLUMNS_RUN,
    "vector_columns": TAG_VECTOR_COLUMNS
}

jaccard_index = BinaryJaccardIndex()
sigmoid = torch.nn.Sigmoid()

average_encodings_df = encodings_df.copy()
# for mul_column in MULTIPLICATION_COLUMNS:
#     average_encodings_df[mul_column + MULTIPLICATION_EXTENSION] = average_encodings_df[mul_column].apply(lambda x : sigmoid(x))
#     DISTANCE_COLUMNS_RUN.append(mul_column + MULTIPLICATION_EXTENSION)


model_correlations = {column[0]:np.empty(shape=(0,len(TAG_VECTOR_COLUMNS))) for column in DISTANCE_COLUMNS_RUN}
for i, row in tqdm(enumerate(average_encodings_df.sample(10, random_state=42).index)):
    for tag_column in TAG_VECTOR_COLUMNS:
        target_vector = average_encodings_df.loc[row][tag_column]
        average_encodings_df[tag_column + VECTOR_DISTANCE_EXTENSION] = average_encodings_df[tag_column].apply(lambda x: F.cosine_similarity(x, target_vector)[0].item())
    
    for evaluation_column in DISTANCE_COLUMNS_RUN:
        if type(evaluation_column) == tuple:
            evaluation_column = evaluation_column[0]
        target_embeddings = average_encodings_df.loc[row][evaluation_column]
        average_encodings_df[evaluation_column + DISTANCE_COLUMN_EXSTENSION] = average_encodings_df[evaluation_column].apply(lambda x: distance_evaluator_altered(target_embeddings, x))
        average_encodings_df[evaluation_column + DISTANCE_COLUMN_EXSTENSION] = average_encodings_df[evaluation_column + DISTANCE_COLUMN_EXSTENSION].apply(lambda x: np.round(x, 4))
    
    closest_df = pd.DataFrame(average_encodings_df.groupby("owner").head(1))[["owner"] + [tag_column + VECTOR_DISTANCE_EXTENSION for tag_column in TAG_VECTOR_COLUMNS]]
    for resnet_column in DISTANCE_COLUMNS_RUN:
        if type(resnet_column) == tuple:
            resnet_column = resnet_column[0]
        column_name = resnet_column + DISTANCE_COLUMN_EXSTENSION
        tmp_df = pd.DataFrame(average_encodings_df.groupby("owner").head(1)).sort_values(by=column_name, ascending=True)[["owner", column_name]]
        closest_df = pd.merge(closest_df, tmp_df, on="owner", how="inner")
        tag_correlations = []
        for tag_column in TAG_VECTOR_COLUMNS:
            #tag_correlations.append(closest_df[tag_column + VECTOR_DISTANCE_EXTENSION].corr(closest_df[column_name]) ** 2)
            tag_correlations.append(closest_df[tag_column + VECTOR_DISTANCE_EXTENSION].corr(closest_df[column_name]))
        model_correlations[resnet_column] = np.vstack([model_correlations[resnet_column], np.array(tag_correlations)])

    if LOG_WANDB:
        wandb_dict = {}
        for column, correlations in model_correlations.items():
            wandb_dict = {**wandb_dict, **{f"{column}_{tag_column}":correlations[i,j] for j, tag_column in enumerate(TAG_VECTOR_COLUMNS)}}
        wandb.log(wandb_dict)

#for column, correlations in model_correlations.items():
for column in DISTANCE_COLUMNS_RUN:
    if type(column) == tuple:
        column, acc_1, acc_5 = column
    else:
        acc_1, acc_5 = -1, -1
    correlations = model_correlations[column]
    combined_accuracy = ((acc_1 + acc_5) / 2) / 100
    correlation_string, combined_accuracy_string = "", ""
    for j, tag_column in enumerate(TAG_VECTOR_COLUMNS):
        correlation_score = np.array(correlations[:,j]).mean()
        correlation_string += f"({tag_column}: {round(correlation_score, 3)})"
        combined_accuracy_string += f"({tag_column}: {round(correlation_score / combined_accuracy, 3)})"
    #print(f"{correlation_string} {column}: Average correlation between {column} and tags")
    print(f"Acc score: {combined_accuracy_string} {column}: Accuracy correlation score {column} and tags, with an accuracy of {round(combined_accuracy, 3)} and a correlation of {round(correlation_score, 5)}")

Evaluating distances using hard-coded column subset


10it [00:41,  4.15s/it]

Acc score: (tag_vector: 21.514)(tag_vector_binary: 20.046) resnet50_removed_mixup_cutmix_model_50: Accuracy correlation score resnet50_removed_mixup_cutmix_model_50 and tags, with an accuracy of 0.01 and a correlation of 0.20046
Acc score: (tag_vector: 22.657)(tag_vector_binary: 21.789) resnet50_removed_mixup_cutmix_model_100: Accuracy correlation score resnet50_removed_mixup_cutmix_model_100 and tags, with an accuracy of 0.01 and a correlation of 0.21789
Acc score: (tag_vector: 18.872)(tag_vector_binary: 18.49) resnet50_removed_mixup_cutmix_model_150: Accuracy correlation score resnet50_removed_mixup_cutmix_model_150 and tags, with an accuracy of 0.01 and a correlation of 0.1849
Acc score: (tag_vector: 19.646)(tag_vector_binary: 18.927) resnet50_removed_mixup_cutmix_model_200: Accuracy correlation score resnet50_removed_mixup_cutmix_model_200 and tags, with an accuracy of 0.01 and a correlation of 0.18927
Acc score: (tag_vector: 23.546)(tag_vector_binary: 23.099) resnet50_removed_mixu